<a href="https://colab.research.google.com/github/melzismn/fmap/blob/master/fmaps_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --force-reinstall Pillow

import igl
import scipy
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import meshplot
from scipy.sparse.linalg import eigs,eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import tensorflow as tf
tf.__version__

In [ ]:
# We will need to plot a pair of shapes
def plot_pair(v1, v2, f1, f2, c1, c2, color_ops = {}):
    # Compute a scale factor
    M1 = igl.massmatrix(v1, f1, igl.MASSMATRIX_TYPE_VORONOI)
    M2 = igl.massmatrix(v2, f2, igl.MASSMATRIX_TYPE_VORONOI)
    scale_factor = np.sqrt(np.sum(M2)/np.sum(M1))

    # Align the shapes
    v2 = v2 - np.mean(v2,axis=0)
    v1_align = v1 * scale_factor + np.mean(v1,axis=0) + [0.7,-0.7,0.0]

    # Merge the models
    v_all = np.vstack((v1_align, v2))
    f_all = np.vstack((f1, f2 + np.max(f1)+1))
    c_all = np.vstack((c1, c2))
    
    plot(v_all, f_all, c_all, shading = color_ops)

In [ ]:
# Load Shapes and compute the LBO eigendecomposition
v_src, f_src = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_089.off"))

L_src = -igl.cotmatrix(v_src, f_src)
M_src = igl.massmatrix(v_src, f_src, igl.MASSMATRIX_TYPE_VORONOI)

try:
    evals_src, evecs_src = eigsh(L_src, 200, M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
except:
    evals_src, evecs_src = eigsh(L_src- 1e-8* scipy.sparse.identity(v_src.shape[0]), 200,
                         M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)

evals_src = evals_src.astype(np.float32)
evals_src = evals_src.astype(np.float32)

v_tar, f_tar = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_090.off"))

L_tar = -igl.cotmatrix(v_tar, f_tar)
M_tar = igl.massmatrix(v_tar, f_tar, igl.MASSMATRIX_TYPE_VORONOI)

try:
    evals_tar, evecs_tar = eigsh(L_tar, 200, M_tar, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
except:
    evals_tar, evecs_tar = eigsh(L_tar- 1e-8* scipy.sparse.identity(v_tar.shape[0]), 200,
                         M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
    
evals_tar = evals_tar.astype(np.float32)
evals_tar = evals_tar.astype(np.float32)

In [ ]:
# compute the WKS descriptor
def WKS(vertices, faces, evals, evecs, wks_size, variance):
    # Number of vertices
    n = vertices.shape[0]
    WKS = np.zeros((n,wks_size))

    # Just for numerical stability
    evals[evals<1e-6] = 1e-6

    # log(E)
    log_E = np.log(evals).T

    # Define the energies step
    e = np.linspace(log_E[1], np.max(log_E)/1.02, wks_size)

    # Compute the sigma
    sigma = (e[1]-e[0]) * variance
    C = np.zeros((wks_size,1))

    for i in np.arange(0,wks_size):
        # Computing WKS
        WKS[:,i] = np.sum(
            (evecs)**2 * np.tile( np.exp((-(e[i] - log_E)**2) / (2*sigma**2)),(n,1)), axis=1)
        
        # Noramlization
        C[i] = np.sum(np.exp((-(e[i]-log_E)**2)/(2*sigma**2)))
        
    WKS = np.divide(WKS,np.tile(C,(1,n)).T)
    return WKS

In [ ]:
# Computing the descriptors for the two shapes
d_src = WKS(v_src, f_src, evals_src, evecs_src, 100, 7)
d_tar = WKS(v_tar, f_tar, evals_tar, evecs_tar, 100, 7)

# Visualizing descriptors
i = 2
plot_pair(v_src, v_tar, f_src, f_tar, d_src[:,i:i+1], d_tar[:,i:i+1])

# Nearest Neighbor to obtain the p2p map
treesearch = sp.spatial.cKDTree(d_tar)
p2p = treesearch.query(d_src, k=1)[1]

# To see the quality of the matching we plot a function on one shape and we transfer it to the other
funz_ = (v_tar - np.min(v_tar,0))/np.tile((np.max(v_tar,0)-np.min(v_tar,0)),(np.size(v_tar,0),1));
colors = np.cos(funz_);
funz_tar = (colors-np.min(colors))/(np.max(colors) - np.min(colors));
funz_src = funz_tar[p2p]

# Compute and plot and (euclidean) error evaluation
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

# Functional Maps implementation

In [ ]:
# Computing descriptors
n_land =  10
n_wks  =  20

n_evals = 20

# Landmarks
step = np.int(np.ceil(v_src.shape[0] / n_land))
a = np.arange(0,v_src.shape[0],step)
landmarks = np.zeros((v_src.shape[0], a.size))
landmarks[a,np.arange(a.size)] = 1

# WKS
d_src = WKS(v_src, f_src, evals_src, evecs_src, n_wks, 7)
d_tar = WKS(v_tar, f_tar, evals_tar, evecs_tar, n_wks, 7)

# Optimization Process
desc_src = np.hstack((landmarks,d_src))
desc_tar = np.hstack((landmarks,d_tar))

# Descriptor normalization
no = np.sqrt(np.diag(np.matmul(M_src.T.__matmul__(desc_src).T, desc_src)))
no_s = np.tile(no.T,(v_src.shape[0],1))
no_t = np.tile(no.T,(v_tar.shape[0],1))
fct_src = np.divide(desc_src,no_s)
fct_tar = np.divide(desc_tar,no_t)

# Coefficents of the obtained descriptors
Fct_src = np.matmul(M_src.T.__matmul__(evecs_src[:, 0:n_evals]).T, fct_src)
Fct_tar = np.matmul(M_tar.T.__matmul__(evecs_tar[:, 0:n_evals]).T, fct_tar)

# The relation between the two constant functions can be computed in a closed form
constFct = np.zeros((n_evals,1))
constFct[0, 0] = np.sign(evecs_src[0, 0] * evecs_tar[0, 0]) * np.sqrt(np.sum(M_tar)/np.sum(M_src))

# Energy weights
a = 1e-1 # Descriptors preservation
c = 1e-8 # Commutativity with Laplacian

# Define tensorflow objects
fs = tf.constant(Fct_src, dtype=tf.float32)
ft = tf.constant(Fct_tar, dtype=tf.float32)
evals = tf.constant(tf.linalg.tensor_diag(np.reshape(np.float32(evals_src[0:n_evals]), (n_evals,))), dtype=tf.float32)
evalt = tf.constant(tf.linalg.tensor_diag(np.reshape(np.float32(evals_tar[0:n_evals]), (n_evals,))), dtype=tf.float32)

# Initialize C
C_ini = np.zeros((n_evals,n_evals))
C_ini[0,0]=constFct[0,0]
C = tf.Variable(tf.zeros((n_evals,n_evals), dtype=tf.float32))
C.assign(C_ini)

# Optimizer
adam = tf.keras.optimizers.Adam(1e-1) # Optimization technique
trainable_vars = [C]

# Optimization
for i in np.arange(0,1000):
    with tf.GradientTape() as tape:
        loss1 = a * tf.reduce_sum(((tf.matmul(C, fs) - ft) ** 2)) / 2 # Descriptor preservation
        loss2 = c * tf.reduce_sum((tf.matmul(C, evals) - tf.matmul(evalt,C))**2) #tf.reduce_sum(((C ** 2) * Dlb) / 2)  # Commute with Laplacian
        #loss3 = 1e-6 *  tf.reduce_sum(tf.square(tf.matmul(tf.transpose(C),C) - tf.eye(n_evals))) # Orthonormal C
        loss = loss1  + loss2# + loss3

    # Apply gradient
    grad = tape.gradient(loss,trainable_vars)
    tmp = adam.apply_gradients(zip(grad,trainable_vars))

C = C.numpy()
plt.imshow(C)
plt.colorbar()
plt.show()

treesearch = sp.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals],C))
p2p = treesearch.query(evecs_src[:,0:n_evals], k=1)[1]
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_src - v_src[p2p,:]))
print(err)

# ICP

In [ ]:
print('ICP refine...')
for k in np.arange(0,5):
    treesearch = scipy.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals],C))
    matches = treesearch.query(evecs_src[:,0:n_evals], k=1)[1]
    W = np.linalg.lstsq(evecs_src[matches, 0:n_evals],evecs_tar[:, 0:n_evals])[0]
    d = np.linalg.svd(W)
    C_ICP = np.matmul(np.matmul(d[0], np.eye(n_evals)), d[2])

plt.imshow(C_ICP)
plt.colorbar()
plt.show()

treesearch = sp.spatial.cKDTree(evecs_tar[:,0:n_evals])
p2p = treesearch.query(np.matmul(evecs_src[:,0:n_evals], C_ICP), k=1)[1]
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

# ZoomOut refinement

In [ ]:
# More iterations => bigger map => higher frequencies => better quality
n_iter = 10

# ZOOMOUT
C_iter = C_ICP
for i in np.arange(0,n_iter):
  # 1) Convert into dense correspondence
  treesearch = sp.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals+i], C_iter.T))
  p2p = treesearch.query(evecs_src[:,0:n_evals+i], k=1)[1]

  #2) Convert into C of dimension (n+1) x (n+1)
  C_iter = np.matmul(np.linalg.pinv(evecs_src[:,0:n_evals+i+1]),evecs_tar[p2p,0:n_evals+i+1])


# Evaluate the map and visualize the result
plt.imshow(C_iter)
plt.colorbar()
plt.show()
treesearch = sp.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals+n_iter],C_iter.T))
p2p = treesearch.query(evecs_src[:,0:n_evals+n_iter], k=1)[1]
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)